# Exchange Seats
### The objective of the code challenge is exchange the seats.<br>

> *For instance person on seat number one will be on seat number two and person in number two on seat number one*





In [0]:

# Input table:
# +------------+--------------+
# |     Seat   |  Name        |
# +------------+--------------+
# | 1          | Samara       |
# | 2          | Bria         |
# | 3          | Leroy        |
# | 4          | Mallory      |
# | 5          | Louis        |
# | 6          | Alessandro   |
# +------------+--------------+


In [0]:
from pyspark.sql.functions import lag, lead, when, col, coalesce
from pyspark.sql.window import Window

In [0]:
data = [(1,'Samara'),(2,'Bria'),(3,'Leroy'),(4,'Mallory'),(5,'Louis'),(6,'Alessandro')]
df = spark.createDataFrame(data,['seat','name'])
display(df)

seat,name
1,Samara
2,Bria
3,Leroy
4,Mallory
5,Louis
6,Alessandro


In [0]:
exchange_seat = df.withColumn('prev_name_seat', lag('name').over(Window.orderBy('seat')))
exchange_seat = exchange_seat.withColumn('next_name_seat', lead('name').over(Window.orderBy('seat')))
display(exchange_seat)

seat,name,prev_name_seat,next_name_seat
1,Samara,null,Bria
2,Bria,Samara,Leroy
3,Leroy,Bria,Mallory
4,Mallory,Leroy,Louis
5,Louis,Mallory,Alessandro
6,Alessandro,Louis,null


In [0]:
exchange_seat = exchange_seat.withColumn('exchanged_seating',
                                         when(exchange_seat['seat'] % 2 == 1, exchange_seat['next_name_seat'] )
                                         .when(exchange_seat['seat'] % 2 == 0, exchange_seat['prev_name_seat'] )
                                         .otherwise(exchange_seat['name'])
                                         )

display(exchange_seat)                                       

seat,name,prev_name_seat,next_name_seat,exchanged_seating
1,Samara,null,Bria,Bria
2,Bria,Samara,Leroy,Samara
3,Leroy,Bria,Mallory,Mallory
4,Mallory,Leroy,Louis,Leroy
5,Louis,Mallory,Alessandro,Alessandro
6,Alessandro,Louis,null,Louis


In [0]:
exchange_seat = exchange_seat.drop('name','prev_name_seat','next_name_seat')
exchange_seat.display()

seat,exchanged_seating
1,Bria
2,Samara
3,Mallory
4,Leroy
5,Alessandro
6,Louis
